In [1]:
from tests.models import Pic, Product, Description

In [ ]:
# Если нужно воспользоваться моделями, прогоняем миграции (если их нет так же создаём)
import sys
from django.core.management import execute_from_command_line
execute_from_command_line(['manage.py', 'migrate', 'tests'])

### **django.db.models.fields.related_descriptors.ManyToManyDescriptor**

In [65]:
Product.sizes  # Возвращает объект ManyToManyDescriptor

In [66]:
Product.sizes.__dict__  # имеет следующие атрибуты, нас интересует rel

{'rel': <ManyToManyRel: tests.product>,
 'field': <django.db.models.fields.related.ManyToManyField: sizes>,
 'reverse': False}

In [67]:
Product.sizes.rel.model
#  из атрибута rel можем получить модель, с которой m2m связь (может понадобится при генерации отчёта)

tests.models.Size

In [68]:
# важно учитывать, что обратная связь в fields подаётся как
# Size.objects.values('product__name')
# Однако доступ к дескриптору получаем через аттрибут field +'_set'
Size.product_set

In [69]:
# а доступ к модели в таком случае
Size.product_set.rel.related_model

tests.models.Product

In [70]:
### **django.db.models.fields.related_descriptors.ReverseManyToOneDescriptor**

In [26]:
Pic.size_set

In [27]:
Pic.size_set.__dict__

{'rel': <ManyToOneRel: tests.size>,
 'field': <django.db.models.fields.related.ForeignKey: picture>}

In [2]:
Pic.size_set.rel.related_model

tests.models.Size

In [30]:
Pic.size_set.field.model

tests.models.Size

In [24]:
Size.objects.select_related('picture').query.__str__()

'SELECT "tests_size"."id", "tests_size"."name", "tests_size"."picture_id", "tests_pic"."id", "tests_pic"."img" FROM "tests_size" LEFT OUTER JOIN "tests_pic" ON ("tests_size"."picture_id" = "tests_pic"."id")'

#### Сверху то что префетчим

#### Снизу то что джоиним

### **django.db.models.fields.related_descriptors.ForwardManyToOneDescriptor**

In [73]:
Product.picture

In [74]:
Product.picture.__dict__  # При наличии ForwardManyToOneDescriptor нам понадобится джоин в sql
# С помощью select_related, потому что на один объект главной модели приходится один объект связанный по
# внешнему ключу, в аргументы select_related должна пойти строка "picture" (название поля у модели)

{'field': <django.db.models.fields.related.ForeignKey: picture>}

In [75]:
# Доступ к модели
Product.picture.field.related_model

tests.models.Pic

### **django.db.models.fields.related_descriptors.ReverseOneToOneDescriptor**

In [76]:
print("Дескриптор: ", Product.description)
print('Его атрибуты: ', Product.description.__dict__)
print("Модель: ", Product.description.related.related_model)

Дескриптор:  <django.db.models.fields.related_descriptors.ReverseOneToOneDescriptor object at 0x7f8d3f3221a0>
Его атрибуты:  {'related': <OneToOneRel: tests.description>}
Модель:  <class 'tests.models.Description'>


### **django.db.models.fields.related_descriptors.ForwardOneToOneDescriptor**

In [77]:
print("Дескриптор: ", Description.product)
print('Его атрибуты: ', Description.product.__dict__)
print("Модель: ", Description.product.field.related_model)

Дескриптор:  <django.db.models.fields.related_descriptors.ForwardOneToOneDescriptor object at 0x7f8d3f321db0>
Его атрибуты:  {'field': <django.db.models.fields.related.OneToOneField: product>}
Модель:  <class 'tests.models.Product'>
